In [1]:
import requests
from getkeys import get_gkey
google_api_key = get_gkey()

def get_coordinates(api_key, address, verbose=False):
    try:
        url = f'https://maps.googleapis.com/maps/api/geocode/json?key={google_api_key}&address={address}'
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Norrmalm, Stockholm, Sweden'
stockholm_center = get_coordinates(google_api_key, address)
latitude = stockholm_center[0]
longitude = stockholm_center[1]
print(f'The geograpical coordinate of {address} are {stockholm_center}.')

The geograpical coordinate of Norrmalm, Stockholm, Sweden are [59.3346342, 18.060526].


In [2]:
import shapely.geometry

import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print(f'Stockholm center longitude={longitude}, latitude={latitude}.')
x, y = lonlat_to_xy(longitude, latitude)
print(f'Stockholm center UTM X={x}, Y={y}')
lo, la = xy_to_lonlat(x, y)
print(f'Stockholm center longitude={lo}, latitude={la}.')

Coordinate transformation check
-------------------------------
Stockholm center longitude=18.060526, latitude=59.3346342.
Stockholm center UTM X=674085.449945552, Y=6581315.456865492
Stockholm center longitude=18.060526, latitude=59.33463420000001.


In [3]:
stockholm_center_x, stockholm_center_y = lonlat_to_xy(longitude, latitude) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = stockholm_center_x - 6000
x_step = 600
y_min = stockholm_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(stockholm_center_x, stockholm_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


In [4]:
import folium

map_stockholm = folium.Map(location=[latitude,longitude], zoom_start=13)
folium.Marker([latitude,longitude], popup='Norrmalm').add_to(map_stockholm)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_berlin) 
    folium.Circle([lat, lon], radius=300, color='#2b0082', fill=False).add_to(map_stockholm)
    #folium.Marker([lat, lon]).add_to(map_berlin)
map_stockholm

In [5]:
def get_address(google_api_key, latitude, longitude, verbose=False):
    try:
        url = f'https://maps.googleapis.com/maps/api/geocode/json?key={google_api_key}&latlng={latitude},{longitude}'
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, stockholm_center[0], stockholm_center[1])

In [6]:
import pickle
import pandas as pd

loaded = False
try:
    with open('locations.pkl', 'rb') as f:
        df_locations = pickle.load(f)
        addresses = [address for address in df_locations['Address']]
        loaded = True
except:
    pass

if loaded == False:
    print('Obtaining location addresses: ', end='')
    addresses = []
    for lat, lon in zip(latitudes, longitudes):
        address = get_address(google_api_key,lat, lon)
        if address is None:
            address = 'NO ADDRESS'
        # address = address.replace(', Ελλάδα', '') # We don't need country part of address
        addresses.append(address)
        print(' .', end='')
    print(' done.')

    df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})
    df_locations.to_pickle('./locations.pkl')   

In [7]:
addresses[0:10]

['Lisebergsvägen 127, 125 35 Älvsjö, Sweden',
 'Sävstaholmsvägen 211, 125 71 Älvsjö, Sweden',
 'Mönsteråsgränd 12A, 122 43 Enskede, Sweden',
 'Viborgska vägen 51, 122 37 Enskede, Sweden',
 'Ålandsvägen 15, 122 37 Enskede, Sweden',
 'Hemmansvägen 30, 122 38 Enskede, Sweden',
 'Lilla gränd 14, 122 62 Enskede, Sweden',
 'Skodonsvägen 16, 126 77 Hägersten, Sweden',
 'Västberga allébro, Västberga allébro, 126 39 Hägersten, Sweden',
 'Älvsåkersgränd 48, 125 72 Älvsjö, Sweden']

In [8]:
import json, requests

food_category = '4d4b7105d754a06374d81259'

cafeterias = ["4bf58dd8d48988d128941735", "4bf58dd8d48988d16d941735", "4bf58dd8d48988d18d941735","4bf58dd8d48988d1e0931735"]

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Sweden', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=50):
    url = 'https://api.foursquare.com/v2/venues/explore'

    params = dict(
    client_id=client_id,
    client_secret=client_secret,
    v='20180724',
    ll=f'{lat},{lon}',
    limit=limit,
    radius=radius,
    categoryId=category
    )
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)

    try:
        results = data['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
        pass
    return venues

In [9]:
from getkeys import get_fsqkey
client_id , client_secret = get_fsqkey()

In [10]:
import pickle

def get_cafes(lats, lons):
    cafeterias = {}
    location_cafes = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, client_id, client_secret, radius=350, limit=50)
        area_cafes = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            # is_cafe, is_spec = is_cafe(venue_categories, specific_filter=cafeterias)
            # if is_cafe:
            x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
            cafe = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, x, y)
            if venue_distance<=300:
                area_cafes.append(cafe)
            cafeterias[venue_id] = cafe
                # if is_italian:
                #     italian_restaurants[venue_id] = restaurant
        location_cafes.append(area_cafes)
        print(' .', end='')
    print(' done.')
    return cafeterias, location_cafes

# Try to load from local file system in case we did this before
cafes = {}
location_cafes = []
loaded = False
try:
    with open('cafes.pkl', 'rb') as f:
        cafes = pickle.load(f)
    with open('location_cafes.pkl', 'rb') as f:
        location_cafes = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    cafes,location_cafes = get_cafes(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('cafes.pkl', 'wb') as f:
        pickle.dump(cafes, f)
    with open('location_cafes.pkl', 'wb') as f:
        pickle.dump(location_cafes, f)

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [11]:
import numpy as np

print('Total number of cafes:', len(cafes))
print('Average number of cafes in neighborhood:', np.array([len(r) for r in location_cafes]).mean())

Total number of cafes: 2048
Average number of cafes in neighborhood: 5.010989010989011


In [12]:
# # import json, requests
# url = 'https://api.foursquare.com/v2/venues/explore'
# limit = 50
# params = dict(
# client_id=client_id,
# client_secret=client_secret,
# v='20180724',
# ll=f'{latitude},{longitude}',
# limit=limit,
# radius=350,
# categoryId={food_category}
# )
# resp = requests.get(url=url, params=params)
# data = json.loads(resp.text)
# data
# # results

In [13]:
# results = data['response']['groups'][0]['items']
# venues = [(item['venue']['id'],
#             item['venue']['name'],
#             get_categories(item['venue']['categories']),
#             (item['venue']['location']['lat'], item['venue']['location']['lng']),
#             format_address(item['venue']['location']),
#             item['venue']['location']['distance']) for item in results]  
# venues